In [ ]:
!pip install nlp
!pip install tensorflow_text

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub 
import tensorflow_text as text
import nlp
from nlp import load_dataset
import json

In [ ]:
bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [ ]:
def get_column_embedding(sentences):
  preprocessed_text = bert_preprocess(sentences)
  sentence_encoding = bert_encoder(preprocessed_text)['pooled_output']
  return sentence_encoding

In [ ]:
def get_word_embedding(sentence):
  preprocessed_text = bert_preprocess(sentence)
  sentence_encoding = bert_encoder(preprocessed_text)
  return sentence_encoding['sequence_output']

In [ ]:
train_dataset  = load_dataset('wikisql', split=nlp.Split.TRAIN)
valid_dataset = load_dataset('wikisql', split=nlp.Split.VALIDATION)

In [ ]:
test_dataset = load_dataset('wikisql', split=nlp.Split.TEST)

In [ ]:
train_length = 500
test_length = 100
questions = []
queries = []
columns = []

for i in range(0, train_length):
  questions.append(train_dataset[i]['question'])
  queries.append(train_dataset[i]['sql']['human_readable'])
  columns.append(train_dataset[i]['table']['header'])

In [ ]:
test_questions = []
test_queries = []
test_columns = []

for i in range(0, test_length):
  test_questions.append(test_dataset[i]['question'])
  test_queries.append(test_dataset[i]['sql']['human_readable'])
  test_columns.append(test_dataset[i]['table']['header'])

In [ ]:
import pandas as pd
df = pd.DataFrame(columns)
df[16] = pd.Series('')


df.fillna('', inplace=True)
df.head(15)
print(df.shape)


(5000, 17)


In [ ]:
import pandas as pd
test_df = pd.DataFrame(test_columns)


test_df.fillna('', inplace=True)
print(test_df.shape)
test_df.head(15)


(1000, 17)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,Player,No.,Nationality,Position,Years in Toronto,School/Club Team,,,,,,,,,,,
1,Player,No.,Nationality,Position,Years in Toronto,School/Club Team,,,,,,,,,,,
2,Player,No.,Nationality,Position,Years in Toronto,School/Club Team,,,,,,,,,,,
3,Player,No.,Nationality,Position,Years in Toronto,School/Club Team,,,,,,,,,,,
4,No,Date,Round,Circuit,Pole Position,Fastest Lap,Race winner,Report,,,,,,,,,
5,No,Date,Round,Circuit,Pole Position,Fastest Lap,Race winner,Report,,,,,,,,,
6,No,Date,Round,Circuit,Pole Position,Fastest Lap,Race winner,Report,,,,,,,,,
7,Pick,Player,Position,Nationality,NHL team,College/junior/club team,,,,,,,,,,,
8,Pick,Player,Position,Nationality,NHL team,College/junior/club team,,,,,,,,,,,
9,Pick,Player,Position,Nationality,NHL team,College/junior/club team,,,,,,,,,,,


In [ ]:
question_df = pd.DataFrame(questions)
question_df.head()

,0
0,Tell me what the notes are for South Australia
1,What is the current series where the new serie...
2,What is the format for South Australia?
3,Name the background colour for the Australian ...
4,how many times is the fuel propulsion is cng?


In [ ]:
def get_questions_row(row):
  row = list(row)
  return row[0]

question_df['encoding'] = question_df.apply(lambda row : get_word_embedding(get_questions_row(row)),axis = 1)

In [ ]:
question_embedding = get_word_embedding(questions)

In [ ]:
question_embedding.shape

TensorShape([500, 128, 768])

In [ ]:
test_question_embedding = get_word_embedding(test_questions)
test_question_embedding.shape

TensorShape([100, 128, 768])

In [ ]:
def get_row(row):
  
  print('*')
  row = list(row)
  return row

df['encoding'] = df.apply(lambda row : get_column_embedding(get_row(row)),axis = 1 )

In [ ]:
def get_row(row):
  
  print('*')
  row = list(row)
  return row

test_df['encoding'] = test_df.apply(lambda row : get_column_embedding(get_row(row)),axis = 1 )

In [ ]:
encoding = list(df['encoding'])

len(encoding)
print(encoding[0].shape)

(11, 768)


In [ ]:
test_encoding = list(test_df['encoding'])

len(test_encoding)
print(test_encoding[0].shape)

(11, 768)


In [ ]:
encoding_tf = tf.convert_to_tensor(encoding)

In [ ]:
test_encoding_tf = tf.convert_to_tensor(test_encoding)
test_encoding_tf.shape

TensorShape([200, 11, 768])

In [ ]:
encoding_tf.shape

TensorShape([500, 11, 768])

In [ ]:
array = encoding_tf.numpy()
array

In [ ]:
test_array = test_encoding_tf.numpy()

In [ ]:
test_x =test_array.tolist()

ts = json.dumps(test_x)

with open("test_colemb.json", "w") as outfile:
    outfile.write(ts)

In [ ]:
x =array.tolist()

s = json.dumps(x)

In [ ]:
with open("colemb.json", "w") as outfile:
    outfile.write(s)

In [ ]:
df.to_csv('column_embeds.csv',index=False)

In [ ]:
column_f = open('colemb.json')

column_data = json.load(column_f)

In [ ]:
test_column_f = open('test_colemb.json')

test_column_data = json.load(test_column_f)

In [ ]:
len(column_data[0][0])

768

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,encoding
0,State/territory,Text/background colour,Format,Current slogan,Current series,Notes,,,,,,"((tf.Tensor(-0.9662499, shape=(), dtype=float3..."
1,State/territory,Text/background colour,Format,Current slogan,Current series,Notes,,,,,,"((tf.Tensor(-0.9662499, shape=(), dtype=float3..."
2,State/territory,Text/background colour,Format,Current slogan,Current series,Notes,,,,,,"((tf.Tensor(-0.9662499, shape=(), dtype=float3..."
3,State/territory,Text/background colour,Format,Current slogan,Current series,Notes,,,,,,"((tf.Tensor(-0.9662499, shape=(), dtype=float3..."
4,Order Year,Manufacturer,Model,Fleet Series (Quantity),Powertrain (Engine/Transmission),Fuel Propulsion,,,,,,"((tf.Tensor(-0.85391146, shape=(), dtype=float..."


# COLUMN ATTENTION


In [ ]:
column_embeddings = tf.convert_to_tensor(column_data)

In [ ]:
test_column_embeddings = tf.convert_to_tensor(test_column_data[0:100])
test_column_embeddings.shape

TensorShape([100, 11, 768])

In [ ]:
column_embeddings.shape

TensorShape([500, 11, 768])

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
question_inputs = keras.Input(shape=(128,768),name='question_embeddings')
column_inputs = keras.Input(shape=(11,768),name='column_embeddings')
Hq = layers.LSTM(100, return_sequences = True)(question_inputs)
WHq = layers.Dense(100, activation='tanh')(Hq)

In [ ]:
Hq.shape

TensorShape([None, 128, 100])

In [ ]:
WHq.shape

TensorShape([None, 128, 100])

In [ ]:
Ecol = layers.LSTM(100)(column_inputs)

In [ ]:
Ecol.shape

TensorShape([None, 100])

In [ ]:
print(Ecol.shape)
WHq = tf.keras.layers.Permute((2,1))(WHq)
print(WHq.shape)

(None, 100)
(None, 100, 128)


In [ ]:
Ecol = tf.keras.layers.Reshape((Ecol.shape[1],1))(Ecol)
Ecol = tf.keras.layers.Permute((2,1))(Ecol)
print(Ecol.shape)

(None, 1, 100)


In [ ]:
v =  tf.keras.layers.Dot(axes=(2, 1))([Ecol, WHq])

In [ ]:
v.shape

TensorShape([None, 1, 128])

In [ ]:
w = tf.keras.layers.Softmax()(v)

In [ ]:
w.shape

TensorShape([None, 1, 128])

In [ ]:
Eq_col =  tf.keras.layers.Dot(axes=(2, 1))([w,Hq])

In [ ]:
Eq_col.shape

TensorShape([None, 1, 100])

In [ ]:
Uq_Eq_col = tf.keras.layers.Dense(100)(Eq_col)

In [ ]:
Uq_Eq_col.shape

TensorShape([None, 1, 100])

In [ ]:
Uc_Ecol = tf.keras.layers.Dense(100)(Ecol)

In [ ]:
Uc_Ecol.shape

TensorShape([None, 1, 100])

In [ ]:
add_layer = tf.keras.layers.Add()([Uc_Ecol,Uq_Eq_col])

In [ ]:
activation_output = tf.keras.layers.Activation('tanh')(add_layer)
activation_output.shape

TensorShape([None, 1, 100])

In [ ]:
dense_layer = tf.keras.layers.Dense(11)(activation_output)
dense_layer.shape

TensorShape([None, 1, 11])

In [ ]:
P_where_col = tf.keras.layers.Softmax()(dense_layer)
P_where_col.shape

TensorShape([None, 1, 11])

In [ ]:
model_final = keras.Model(inputs=[column_inputs,question_inputs],outputs = P_where_col)
model_final.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 column_embeddings (InputLayer)  [(None, 11, 768)]   0           []                               
                                                                                                  
 question_embeddings (InputLaye  [(None, 128, 768)]  0           []                               
 r)                                                                                               
                                                                                                  
 lstm_3 (LSTM)                  (None, 100)          347600      ['column_embeddings[0][0]']      
                                                                                                  
 lstm_2 (LSTM)                  (None, 128, 100)     347600      ['question_embeddings[0][0]

In [ ]:
model_final.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
Y_train = []

for i in range(0,500):
  sel_column = [0,0,0,0,0,0,0,0,0,0,0]
  index = train_dataset[i]['sql']['sel']
  sel_column[index] = 1
  Y_train.append(sel_column)

len(Y_train[0])

Y_train

[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0,

In [ ]:
Y_test = []

for i in range(0,100):
  sel_column = [0,0,0,0,0,0,0,0,0,0,0]
  index = test_dataset[i]['sql']['sel']
  sel_column[index] = 1
  Y_test.append(sel_column)

len(Y_test[0])
Y_test = tf.convert_to_tensor(Y_test)

Y_test = tf.reshape(Y_test,[100,1,11])
Y_test.shape

TensorShape([100, 1, 11])

In [ ]:
Y_train = tf.convert_to_tensor(Y_train)

Y_train = tf.reshape(Y_train,[500,1,11])

Y_train.shape

TensorShape([500, 1, 11])

In [ ]:
model_final.fit(
    {"question_embeddings": question_embedding, "column_embeddings": column_embeddings},
    Y_train,
    epochs=30,
    batch_size=16,
)

Epoch 1/30
32/32 [==============================] - 12s 200ms/step - loss: 0.2832 - accuracy: 0.1660
Epoch 2/30
32/32 [==============================] - 6s 200ms/step - loss: 0.2629 - accuracy: 0.2840
Epoch 3/30
32/32 [==============================] - 6s 195ms/step - loss: 0.2288 - accuracy: 0.4300
Epoch 4/30
32/32 [==============================] - 6s 201ms/step - loss: 0.1888 - accuracy: 0.5460
Epoch 5/30
32/32 [==============================] - 6s 195ms/step - loss: 0.1250 - accuracy: 0.7400
Epoch 6/30
32/32 [==============================] - 6s 196ms/step - loss: 0.0859 - accuracy: 0.8320
Epoch 7/30
32/32 [==============================] - 6s 197ms/step - loss: 0.0532 - accuracy: 0.9100
Epoch 8/30
32/32 [==============================] - 6s 198ms/step - loss: 0.0292 - accuracy: 0.9620
Epoch 9/30
32/32 [==============================] - 6s 196ms/step - loss: 0.0162 - accuracy: 0.9920
Epoch 10/30
32/32 [==============================] - 6s 197ms/step - loss: 0.0060 - accuracy: 0.998

In [ ]:
loss, accuracy = model_final.evaluate({"question_embeddings": test_question_embedding, "column_embeddings": test_column_embeddings},Y_test)
print('Test accuracy :', accuracy)

4/4 [==============================] - 1s 78ms/step - loss: 0.5222 - accuracy: 0.2500
Test accuracy : 0.25


# SELECT AGGREGATION

In [ ]:
question_inputs = keras.Input(shape=(128,768),name='question_embeddings_agg')
column_inputs = keras.Input(shape=(11,768),name='column_embeddings_agg')

Hq = layers.Bidirectional(layers.LSTM(50, return_sequences = True))(question_inputs)
WHq = layers.Dense(100, activation='tanh')(Hq)

In [ ]:
Ecol = layers.Bidirectional(layers.LSTM(50))(column_inputs)

In [ ]:
WHq = tf.keras.layers.Permute((2,1))(WHq)
Ecol = tf.keras.layers.Reshape((Ecol.shape[1],1))(Ecol)
Ecol = tf.keras.layers.Permute((2,1))(Ecol)

In [ ]:
v =  tf.keras.layers.Dot(axes=(2, 1))([Ecol, WHq])

In [ ]:
w = tf.keras.layers.Softmax()(v)

In [ ]:
Eq_col =  tf.keras.layers.Dot(axes=(2, 1))([w,Hq])

In [ ]:
Uq_Eq_col = tf.keras.layers.Dense(100)(Eq_col)

In [ ]:
activation_output = tf.keras.layers.Activation('tanh')(Uq_Eq_col)

In [ ]:
dense_layer = tf.keras.layers.Dense(6)(activation_output)

In [ ]:
P_sel_agg = tf.keras.layers.Softmax()(dense_layer)

In [ ]:
model_agg = keras.Model(inputs=[column_inputs,question_inputs],outputs = P_sel_agg)
model_agg.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 column_embeddings_agg (InputLa  [(None, 11, 768)]   0           []                               
 yer)                                                                                             
                                                                                                  
 question_embeddings_agg (Input  [(None, 128, 768)]  0           []                               
 Layer)                                                                                           
                                                                                                  
 bidirectional_3 (Bidirectional  (None, 100)         327600      ['column_embeddings_agg[0][0]']  
 )                                                                                          

In [ ]:
model_agg.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# WHERE NUMBER

In [ ]:
question_inputs = keras.Input(shape=(128,768),name='question_embeddings_where_no')

In [ ]:
Hq = layers.Bidirectional(layers.LSTM(50, return_sequences = True))(question_inputs)
WHq = layers.Dense(100, activation='tanh')(Hq)

In [ ]:
WHq.shape

TensorShape([None, 128, 100])

In [ ]:
Eq = layers.Bidirectional(layers.LSTM(50))(question_inputs)
Eq.shape

TensorShape([None, 100])

In [ ]:
Eq = tf.keras.layers.Reshape((Eq.shape[1],1))(Eq)
Eq.shape

TensorShape([None, 100, 1])

In [ ]:
WHq = tf.keras.layers.Permute((2,1))(WHq)
Eq = tf.keras.layers.Permute((2,1))(Eq)
print(Eq.shape)
print(WHq.shape)

(None, 1, 100)
(None, 100, 128)


In [ ]:
v = tf.keras.layers.Dot(axes=(2, 1))([Eq, WHq])
v.shape

TensorShape([None, 1, 128])

In [ ]:
w = layers.Softmax()(v)
w.shape

TensorShape([None, 1, 128])

In [ ]:
Eqq = tf.keras.layers.Dot(axes=(2, 1))([w, Hq])
Eqq.shape

TensorShape([None, 1, 100])

In [ ]:
U2_Eqq = layers.Dense(100)(Eqq)
U2_Eqq.shape

TensorShape([None, 1, 100])

In [ ]:
tanh_U2_Eqq = layers.Activation('tanh')(U2_Eqq)
tanh_U2_Eqq.shape

TensorShape([None, 1, 100])

In [ ]:
U1_tanh = layers.Dense(12)(tanh_U2_Eqq)
U1_tanh.shape

TensorShape([None, 1, 12])

In [ ]:
P_no_where = layers.Softmax()(U1_tanh)
P_no_where.shape

TensorShape([None, 1, 12])

In [ ]:
model_where_no = keras.Model(inputs=question_inputs,outputs = P_no_where)
model_where_no.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 question_embeddings_where_no (  [(None, 128, 768)]  0           []                               
 InputLayer)                                                                                      
                                                                                                  
 bidirectional_5 (Bidirectional  (None, 100)         327600      ['question_embeddings_where_no[0]
 )                                                               [0]']                            
                                                                                                  
 bidirectional_4 (Bidirectional  (None, 128, 100)    327600      ['question_embeddings_where_no[0]
 )                                                               [0]']                      

In [ ]:
model_where_no.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# WHERE OPERATION

In [ ]:
question_inputs = keras.Input(shape=(128,768),name='question_embeddings_where_op')
column_inputs = keras.Input(shape=(11,768),name='column_embeddings_where_op')

Hq = layers.Bidirectional(layers.LSTM(50, return_sequences = True))(question_inputs)
WHq = layers.Dense(100, activation='tanh')(Hq)

print((Hq.shape))
print(WHq.shape)

(None, 128, 100)
(None, 128, 100)


In [ ]:
Ecol = layers.Bidirectional(layers.LSTM(50,return_sequences = True))(column_inputs)

In [ ]:
WHq = tf.keras.layers.Permute((2,1))(WHq)
# Ecol = tf.keras.layers.Reshape((Ecol.shape[1],1))(Ecol)
# Ecol = tf.keras.layers.Permute((2,1))(Ecol)

print(WHq.shape)
print(Ecol.shape)

(None, 100, 128)
(None, 11, 100)


In [ ]:
v =  tf.keras.layers.Dot(axes=(2, 1))([Ecol, WHq])
print(v.shape)

(None, 11, 128)


In [ ]:
w = tf.keras.layers.Softmax()(v)

In [ ]:
Eq_col =  tf.keras.layers.Dot(axes=(2, 1))([w,Hq])
print(Eq_col.shape)

(None, 11, 100)


In [ ]:
Uq_Eq_col = tf.keras.layers.Dense(100)(Eq_col)
Uq_Eq_col.shape

TensorShape([None, 11, 100])

In [ ]:
Uc_Ecol = tf.keras.layers.Dense(100)(Ecol)
Uc_Ecol.shape

TensorShape([None, 11, 100])

In [ ]:
add_layer = tf.keras.layers.Add()([Uc_Ecol,Uq_Eq_col])
add_layer.shape

TensorShape([None, 11, 100])

In [ ]:
activation_output = tf.keras.layers.Activation('tanh')(Uq_Eq_col)

In [ ]:
dense_layer = tf.keras.layers.Dense(4)(activation_output)
dense_layer.shape

TensorShape([None, 11, 4])

In [ ]:
P_where_ops = tf.keras.layers.Softmax()(dense_layer)
P_where_ops.shape

TensorShape([None, 11, 4])

In [ ]:
model_where_ops = keras.Model(inputs=[question_inputs,column_inputs],outputs = P_where_ops)
model_where_ops.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 question_embeddings_where_op (  [(None, 128, 768)]  0           []                               
 InputLayer)                                                                                      
                                                                                                  
 bidirectional (Bidirectional)  (None, 128, 100)     327600      ['question_embeddings_where_op[0]
                                                                 [0]']                            
                                                                                                  
 column_embeddings_where_op (In  [(None, 11, 768)]   0           []                               
 putLayer)                                                                                    

In [ ]:
model_where_ops.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# WHERE VALUE

In [ ]:
encoder_inputs = layers.Input(shape=(128,768 ),name='question_inputs_where_val' ,)
encoder_LSTM = layers.Bidirectional(layers.LSTM(50, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_LSTM(encoder_inputs)
state_h = layers.Concatenate()([forward_h, backward_h])
state_c = layers.Concatenate()([forward_c, backward_c])
print(encoder_outputs.shape)
print(state_h.shape)
print(state_c.shape)


(None, 100)
(None, 100)
(None, 100)
